# `metadsl`

...is a framework for creating domain specific languages in Python.

In [1]:
from __future__ import annotations

from metadsl import *
from metadsl_core import *
from metadsl_visualize import *

Here, we show a NumPy API compatible DSL that we have started writing. We can create a symbolic expression and then execute it. The widget shows the different stages of execution:

In [2]:
N = 100
expr = (arange(N) + arange(N))[5].to_ndarray()
expr

Typez(definitions=None, nodes={'2436643635173622306': PrimitiveNode(type='int', repr='100'), '-3079624557439639756': CallNode(function='metadsl_core.numpy.arange', type_params=None, args=['2436643635173622306'], kwargs=None), '-6910758923161075491': CallNode(function='metadsl_core.numpy.NDArrayCompat.__add__', type_params=None, args=['-3079624557439639756', '-3079624557439639756'], kwargs=None), '2436643635276462181': PrimitiveNode(type='int', repr='5'), '8324735090661096784': CallNode(function='metadsl_core.numpy.NDArrayCompat.__getitem__', type_params=None, args=['-6910758923161075491', '2436643635276462181'], kwargs=None), '-4297988362652024148': CallNode(function='metadsl_core.numpy.NDArrayCompat.to_ndarray', type_params=None, args=['8324735090661096784'], kwargs=None), '-2571194992384755768': CallNode(function='metadsl_core.conversion.Converter.convert', type_params={'T': DeclaredTypeInstance(type='metadsl_core.numpy.NDArray', params=None)}, args=['-6910758923161075491'], kwargs=N

replaced typevars.

original:
<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__getitem__, (<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__add__, (<class 'metadsl_core.numpy.NDArrayCompat'>(arange, (100,), {}), <class 'metadsl_core.numpy.NDArrayCompat'>(arange, (100,), {})), {}), 5), {})

old:
<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.from_ndarray, (metadsl_core.maybe.Maybe[metadsl_core.numpy.NDArray](metadsl_core.maybe.Maybe[metadsl_core.pair.Pair[metadsl_core.numpy.NDArray, metadsl_core.either.Either[metadsl_core.integer.Integer, metadsl_core.vec.Vec[metadsl_core.integer.Integer]]]].map, (metadsl_core.maybe.Maybe[metadsl_core.pair.Pair[metadsl_core.numpy.NDArray, metadsl_core.either.Either[metadsl_core.integer.Integer, metadsl_core.vec.Vec[metadsl_core.integer.Integer]]]](metadsl_core.maybe.Maybe[metadsl_core.numpy.NDArray].__and__, (metadsl_core.maybe.Maybe[metadsl_core.numpy.NDArray](metadsl_core.conversion.Converter[metadsl_core.numpy.NDArray].c

10

Now, we can add an optimization replacement, so that we do the getitem before adding the two expressions:

In [3]:
@register
@rule
def optimize_getitem_add(l: NDArray, r: NDArray, idx: IndxType) -> R[NDArray]:
    return (
        # expression to match against
        (l + r)[idx],
        # expression to replace it with
        l[idx] + r[idx]
    )

Now, when we look at the execution steps, we see that it is doing each getitem before adding, which should be more efficient:

In [4]:
expr = (arange(N) + arange(N))[5].to_ndarray()
expr

Typez(definitions=None, nodes={'2436643635173622306': PrimitiveNode(type='int', repr='100'), '-3079624557439639756': CallNode(function='metadsl_core.numpy.arange', type_params=None, args=['2436643635173622306'], kwargs=None), '-6910758923161075491': CallNode(function='metadsl_core.numpy.NDArrayCompat.__add__', type_params=None, args=['-3079624557439639756', '-3079624557439639756'], kwargs=None), '2436643635276462181': PrimitiveNode(type='int', repr='5'), '8324735090661096784': CallNode(function='metadsl_core.numpy.NDArrayCompat.__getitem__', type_params=None, args=['-6910758923161075491', '2436643635276462181'], kwargs=None), '-4297988362652024148': CallNode(function='metadsl_core.numpy.NDArrayCompat.to_ndarray', type_params=None, args=['8324735090661096784'], kwargs=None), '-2571194992384755768': CallNode(function='metadsl_core.conversion.Converter.convert', type_params={'T': DeclaredTypeInstance(type='metadsl_core.numpy.NDArray', params=None)}, args=['-6910758923161075491'], kwargs=N

replaced typevars.

original:
<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__getitem__, (<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__add__, (<class 'metadsl_core.numpy.NDArrayCompat'>(arange, (100,), {}), <class 'metadsl_core.numpy.NDArrayCompat'>(arange, (100,), {})), {}), 5), {})

old:
<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.from_ndarray, (metadsl_core.maybe.Maybe[metadsl_core.numpy.NDArray](metadsl_core.maybe.Maybe[metadsl_core.pair.Pair[metadsl_core.numpy.NDArray, metadsl_core.either.Either[metadsl_core.integer.Integer, metadsl_core.vec.Vec[metadsl_core.integer.Integer]]]].map, (metadsl_core.maybe.Maybe[metadsl_core.pair.Pair[metadsl_core.numpy.NDArray, metadsl_core.either.Either[metadsl_core.integer.Integer, metadsl_core.vec.Vec[metadsl_core.integer.Integer]]]](metadsl_core.maybe.Maybe[metadsl_core.numpy.NDArray].__and__, (metadsl_core.maybe.Maybe[metadsl_core.numpy.NDArray](metadsl_core.conversion.Converter[metadsl_core.numpy.NDArray].c

10

This shows how we can break up the NumPy API into different layers, all of which are extensible:

1. A compatibility layer that works like the existing NumPy API, except isn't limited to the Python types of the current API
2. A type safe version of this API. The conversion between the compatability layer and this layer is extensible, so that third party authors can add new conversion between their own Python objects and the typed representation.
3. A backend layer that translates either back to Python calls or source code, or to other targets like LLVM or Tensorflow.

The key is that all these layers are composable, so you could have different frontends for any of them or add your own. This is all done through a typed replacement system that is compatible with static analysis using MyPy.

`metadsl` is the glue that allows us to define this API in a way that enables greater greater collaboration. Since each layer has it's own well defined specification, different projects can extend and target these seperate layers, while minimizing the need to explicitly opt in to collaborate. The goal is to enable greater reuse, interoperability, and extensability within the Python scientific ecosystem.